In [1]:
import pandas as pd
dataset = pd.read_csv('NBA.csv',parse_dates=["Date"])
dataset.drop(columns=["Start (ET)","Attend."],axis=1,inplace=True)


dataset.columns = ["Date", "Visitor Team", "VisitorPts",
 "Home Team", "HomePts","Score Type", "OT?", "Notes"]

dataset= dataset.reindex(columns=["Date", "Score Type", "Visitor Team",
"VisitorPts", "Home Team", "HomePts", "OT?", "Notes"])

dataset[:5]

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
1,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
2,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
4,2013-10-30,Box Score,Boston Celtics,87,Toronto Raptors,93,NaN,NaN


In [2]:
dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]
y_true = dataset["HomeWin"].values
print(y_true)

[ True  True  True  True  True  True  True  True False  True  True  True
  True False  True  True  True  True  True]


In [6]:
from collections import defaultdict
won_last = defaultdict(bool)
dataset["HomeLastWin"] = False
dataset["VisitorLastWin"] = False
for index, row in dataset.iterrows():
    
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    dataset.loc[index] = row
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]
dataset[:]

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,False,False
1,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,False,False
2,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,False,False
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,False,False
4,2013-10-30,Box Score,Boston Celtics,87,Toronto Raptors,93,NaN,NaN,True,False,False
5,2013-10-30,Box Score,Miami Heat,110,Philadelphia 76ers,114,NaN,NaN,True,False,True
6,2013-10-30,Box Score,Washington Wizards,102,Detroit Pistons,113,NaN,NaN,True,False,False
7,2013-10-30,Box Score,Milwaukee Bucks,83,New York Knicks,90,NaN,NaN,True,False,False
8,2013-10-30,Box Score,Indiana Pacers,95,New Orleans Pelicans,90,NaN,NaN,False,False,True
9,2013-10-30,Box Score,Orlando Magic,115,Minnesota Timberwolves,120,OT,NaN,True,False,False


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=5)

In [18]:
from sklearn.model_selection import cross_val_score
import numpy as np
X_previouswins = dataset[["HomeLastWin", "VisitorLastWin"]].values

scores = cross_val_score(clf, X_previouswins, y_true,scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))
print(scores)

Accuracy: 70.0%
[1.   1.   0.75 0.75 0.  ]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
